In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
%matplotlib inline

In [3]:
np.random.seed(2355478)
tmp = np.random.poisson(lam=57.3, size=365*10*4)
tmp[np.random.choice(np.arange(tmp.shape[0]),size=365*32, replace=False)] = 0
df = pd.DataFrame(tmp.reshape((40,365)),
                  index=pd.MultiIndex.from_product([np.arange(1,11),np.arange(4)], names=['Customer','Product']),
                  columns=pd.date_range(start='2017-01-01', end='2017-12-31'))

# Assignment 1
The DataFrame `df` holds the customers spend on a daily basis for a number of products. The column names are the individual dates in 2017, while the index values are the combinations of customer IDs and product IDs.

Find the average amount spent by a customer between 2017-04-02 and 2017-08-17, both dates included.
<details><summary>Hint 1</summary>
    <p>
        Entering a start and end date separated by `:` in the `loc` method can be used to select a range of dates. Each date should be a string in the `YYYY-MM-DD` format
    </p>
</details>
<details><summary>Hint 2</summary>
    <p>
        Grouping by `Customer` and then summing twice, once over products and once of dates, gives the total amount spent for each customer in the time period
    </p>
</details>

In [4]:
#for the mean for each month by customer
#df.loc[(1,0):,'2017-04-02':'2017-08-17'].groupby(by = 'Customer').mean()

#for the mean for each customer between 2017-04-02 and 2017-08-17
df_mean=df.loc[(1,0):,'2017-04-02':'2017-08-17'].groupby(by = 'Customer').mean()
df_mean.mean(axis=1)

Customer
1     12.159420
2      9.199275
3     11.663043
4     11.547101
5     13.637681
6     12.193841
7     11.771739
8      9.927536
9     11.751812
10    11.067029
dtype: float64

# Assignment 2
Using the DataFrame `df` again, find the total monthly spend per product. Return a DataFrame where the index is the product ID and the columns are the month number
<details><summary>Hint 1</summary>
    <p>
        A specific aggregation function can be set for the DataFrame method `pivot_table`.
    </p>
</details>
<details><summary>Hint 2</summary>
    <p>
        The `melt` method can transform a DataFrame from a wide format to a long format, making it easier manipulate multiple columns at the same time
    </p>
</details>
<details><summary>Hint 3</summary>
    <p>
        The `id_vars` parameter can be set to prevent some columns from being melted
    </p>
</details>
<details><summary>Hint 4</summary>
    <p>
        The `dt.month` attribute can be used to extract the month from a column containing datetime values
    </p>
</details>

In [5]:
pd.pivot_table(df,index=['Product'],aggfunc=np.sum)

,2017-01-01 00:00:00,2017-01-02 00:00:00,2017-01-03 00:00:00,2017-01-04 00:00:00,2017-01-05 00:00:00,2017-01-06 00:00:00,2017-01-07 00:00:00,2017-01-08 00:00:00,2017-01-09 00:00:00,2017-01-10 00:00:00,...,2017-12-22 00:00:00,2017-12-23 00:00:00,2017-12-24 00:00:00,2017-12-25 00:00:00,2017-12-26 00:00:00,2017-12-27 00:00:00,2017-12-28 00:00:00,2017-12-29 00:00:00,2017-12-30 00:00:00,2017-12-31 00:00:00
Product,,,,,,,,,,,,,,,,,,,,,
0,146,53,77,0,57,162,188,109,243,106,...,62,200,117,100,112,168,59,59,90,109
1,0,309,114,184,67,45,116,104,174,130,...,63,0,0,171,61,140,159,107,221,114
2,62,238,170,57,111,55,177,112,104,62,...,105,115,191,159,62,106,58,176,0,109
3,52,121,221,52,106,163,111,136,234,50,...,106,65,52,0,48,183,247,0,111,58


# Assignment 3
Find the average spend of the first three transactions (days with spend larger than zero) on a product for each customer. Create a DataFrame with the result, where the Customer ID is the index and the Product ID is the column name
<details><summary>Hint 1</summary>
    <p>
        The `head` method can be applied to a GroupBy object to get the first records for every unique combination of the grouping variables
    </p>
</details>
<details><summary>Hint 2</summary>
    <p>
        After melting the DataFrame the days without transactions can be removed by only taking the rows with `value` > 0
    </p>
</details>
<details><summary>Hint 3</summary>
    <p>
        The default aggregation function for the `pivot_table` method is a mean over the values
    </p>
</details>

In [42]:
df_answer3 = df.reset_index()
df_answer3 =df_answer3.melt(id_vars=['Customer', 'Product'])
df_answer3 = df_answer3[df_answer3.value > 0]
pd.pivot_table(df_answer3.groupby(['Customer', 'Product']).head(3), index=['Customer', 'Product'])



value
Customer Product           
1        0        60.000000
         1        66.333333
         2        60.000000
         3        55.666667
2        0        51.333333
         1        56.000000
         2        58.333333
         3        54.333333
3        0        53.000000
         1        57.000000
         2        54.666667
         3        54.000000
4        0        65.333333
         1        47.333333
         2        63.333333
         3        60.666667
5        0        61.333333
         1        57.666667
         2        50.666667
         3        51.000000
6        0        56.000000
         1        69.333333
         2        54.666667
         3        53.000000
7        0        49.000000
         1        52.666667
         2        55.000000
         3        59.333333
8        0        59.333333
         1        52.666667
         2        56.000000
         3        56.000000
9        0        57.000000
         1        60.666667
         2        57.666667
         3        55.666667
10       0        49.666667
         1        56.333333
         2        61.000000
         3        62.333333

# Assignment 4
Find the average days between purchases of a product for each customer. Return the result as a DataFrame with the Customer ID as index and the Product ID as column name
<details><summary>Hint 1</summary>
    <p>
        The `shift` method for a pandas Series moves all values up or down according to the parameter passed to it
    </p>
</details>
<details><summary>Hint 2</summary>
    <p>
        Using the `apply` method on a GroupBy object makes it possible to apply a function to the DataFrame made up of the records in each of the groups
    </p>
</details>
<details><summary>Hint 3</summary>
    <p>
        The DataFrame method `pivot` is a short-hand version of `pivot_table`, that does not support aggregation. Serves as a check that the assumption of uniquenes in the combination of row and column index is valid
    </p>
</details>

# Assignment 5
Create a DataFrame showing the product ID, where a customer has spent the greatest amount of money for each month. Set the Customer IDs as index and the month number as columns. 
<details><summary>Hint 1</summary>
    <p>
        Calculate how much each customer has spent on a product in a given month
    </p>
</details>
<details><summary>Hint 2</summary>
    <p>
        Find the maximum spend for a customer in a month and join it onto the result from the previous hint
    </p>
</details>
<details><summary>Hint 3</summary>
    <p>
        Use the maximum spend to filter the DataFrame to one row for every unique combination of `Customer` and `month`, before applying the `pivot` method
    </p>
</details>